![DLI Header](../images/DLI_Header.png)

# Application Overview: 2D Unsteady Heat Equation

In this notebook you will get familiar with the non-accelerated 2D unsteady heat equation application provided to you as a starting point for your work in this course.

## Learning Objectives

By the time you complete this notebook you should:

- Be familiar with the non-accelerated 2D unsteady heat equation application provided to you as a starting point for your work.
- Be able to compile and run the application.
- Be able to visualize the results of the application.
- Understand the application's output and visualization.

---

## 2D Unsteady Heat Equation

In this directory you will find [starting_point.cpp](./starting_point.cpp) which is a fully functional MPI mini-application. As it stands, the application is not written to run on GPU accelerators, but throughout the course you will be refactoring it using C++ parallel algorithms to obtain an application that can run on multiple nodes, using one GPU per MPI rank.

We gave some coverage to this application in the presentation earlier in this course, but spend some time now familiarizing yourself with [starting_point.cpp](./starting_point.cpp).

---

## Compiling and Running

Run the cell below to compile and run the starting point application.

In [ ]:
!OMPI_CXX=g++ mpicxx -std=c++20 -Ofast -march=native -o heat starting_point.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000

### Compiler Selection

Here we used the `OMPI_CXX` environment variable preceding the `mpicxx` command to pick the C++ compiler, e.g., `OMPI_CXX=g++` picks the GNU g++ compiler. Throughout the course we will be running our application with a variety of compilers.

### Number of Ranks

We will be running the application with 2 MPI ranks. The system you are working on today only contains one node with one GPU, so we will be unable to directly observe the application scaling to multiple nodes, each with their own GPU. However, by running the application with 2 MPI ranks it will require us, as we refactor the application, to maintain its correctness when being run in multiple MPI ranks.

### Program Command Line Interface

* `./heat 256 256 16000`: runs the heat-equation mini-application; it takes three arguments: `NX NY NITERATIONS`
  * `NX` and `NY`: number of unknown per MPI rank in `x` and `y` dimensions
  * `NITERATIONS`: number of time-step to simulate

### Performance Results

This mini-application performs a "weak scaling": `NX` and `NY` control the number of unknowns per MPI rank, such that NX=256 NY=256 results in a grid size of 256x256 nodes per rank. When running the application with, e.g., two ranks, the size of the whole grid doubles to 512x256 along the NX dimension (the NY dimension is kept constant).

The performance results show two memory bandwidths (BWs): the memory BW delivered by 1 rank, and by all ranks.

---

## Visualizing the Application Results

The `heat` binary writes a solution to an `output` file. Assuming you ran the application via the cell above, you should be able to see `output` in the JupyterLab file viewer. The code immediately below can convert `output` to a PNG file which we can use to visualize the results of having run the application.

Execute the cell below to define the `visualize` function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#plt.style.use('dark_background') # Uncomment for dark background

def visualize(name = 'output'):
    f = open(name, 'rb')
    grid = np.fromfile(f, dtype=np.uint64, count=2, offset=0)

    nx = grid[0]
    ny = grid[1]

    times = np.fromfile(f, dtype=np.float64, count=1, offset=0)
    time = times[0]

    values = np.fromfile(f, dtype=np.float64, offset=0)
    assert len(values) == nx * ny, f'{len(values)} != {nx * ny}'
    values = values.reshape((nx, ny))

    print(f'Plotting grid {nx}x{ny}, t = {time}')

    plt.title(f'Temperature at t = {time:.3f} [s]')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.pcolormesh(values, cmap=plt.cm.jet, vmin=0, vmax=values.max())
    plt.colorbar()
    plt.savefig('output.png', transparent=True, bbox_inches='tight', dpi=300)

Execute the following cell to call `visualize` and view a visualized rendering of the output of the `heat` application.

In [ ]:
visualize()

---

## Compiling and Running with `clang++` and `nvc++`

The following 2 cells will compile, run, and visualize the starting point application with `clang++` and `nvc++` respectively. Run the 2 cells now. Confirm the application is running as expecting via the visualization, and make note of the bandwidths reported by the outputs.

Worth mentioning is that the `nvc++` compilation is not yet targeting the application for GPU acceleration.

In [ ]:
!OMPI_CXX=clang++ mpicxx -std=c++20 -Ofast -march=native -o heat starting_point.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

In [ ]:
!OMPI_CXX=nvc++ mpicxx -std=c++20 -O4 -fast -march=native -Mllvm-fast -o heat starting_point.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

---

## Next

Please proceed to [the next notebook](../03-Parallelize/Parallelize.ipynb).

![DLI Header](../images/DLI_Header.png)